<a href="https://colab.research.google.com/github/luishpinto/text-classification/blob/master/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  import github
except:
  !pip install PyGithub
  import github

In [2]:
import os.path

In [3]:
import numpy as np
import pandas as pd

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.pipeline import Pipeline

In [5]:
g = github.Github('luishpinto','eigHt33n')
repo = g.get_user().get_repo('text-classification')
files = np.array([],dtype = str)
for i in repo.get_contents('/data-base'):
  files = np.append(files,i.name)

for i in files:
  if not os.path.exists(i):
    !wget -O {i} 'https://raw.githubusercontent.com/luishpinto/text-classification/master/data-base/{i}'

In [6]:
flist = dict()
flist['yelp'] = 'yelp_labelled.txt'
flist['imdb'] = 'imdb_labelled.txt'
flist['amazon'] = 'amazon_cells_labelled.txt'

In [7]:
dflist = []
for source,path in flist.items():
  df = pd.read_csv(path,names = ['sentence','label'],sep = '\t')
  df['source'] = source
  dflist.append(df)

df = pd.concat(dflist)
print(df.iloc[0])

sentence    Wow... Loved this place.
label                              1
source                          yelp
Name: 0, dtype: object


In [8]:
sentences = ['John likes ice cream','John hates chocolate.']
vectorizer = CountVectorizer(min_df = 0,lowercase = False)
vectorizer.fit(sentences)
vectorizer.vocabulary_

{'John': 0, 'chocolate': 1, 'cream': 2, 'hates': 3, 'ice': 4, 'likes': 5}

In [9]:
vectorizer.transform(sentences).toarray()

array([[1, 0, 1, 0, 1, 1],
       [1, 1, 0, 1, 0, 0]])

In [10]:
classifiers = [LogisticRegression(),
               DecisionTreeClassifier(),
               SGDClassifier()]

for clf in classifiers:

  print('\n\nClassifier: {}\n'.format(clf))

  for i in df['source'].unique():
    dfsource = df[df['source'] == i]
    X = dfsource['sentence'].values
    y = dfsource['label'].values

    Strain,Stest,ytrain,ytest = train_test_split(X,y,test_size = 0.25,random_state = 1000)

    vectorizer = CountVectorizer()
    vectorizer.fit(Strain)

    Xtrain = vectorizer.transform(Strain)
    Xtest = vectorizer.transform(Stest)

    pipe = Pipeline(steps = [('classifier',clf)])
    pipe.fit(Xtrain,ytrain)
    score = pipe.score(Xtest,ytest)

    print('Accuracy for {} dataset: {:.3f}'.format(i,score))



Classifier: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Accuracy for yelp dataset: 0.796
Accuracy for imdb dataset: 0.749
Accuracy for amazon dataset: 0.796


Classifier: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

Accuracy for yelp dataset: 0.772
Accuracy for imdb dataset: 0.626
Accuracy for amazon dataset: 0.784


Classifier: S